In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import datetime as dt
import uuid
from selenium.common.exceptions import StaleElementReferenceException
from tqdm import tqdm

In [2]:
url = 'https://line4bet.ru/1x-13-03-2023-football/'

In [3]:
driver = webdriver.Chrome()
driver.get(url)
assert 'Движение линий' in driver.title

In [4]:
# "Теннис" button searching
tennis_btn = driver.find_elements(By.ID, 'tennis_btn')
assert len(tennis_btn) > 0
tennis_btn[0].click()

In [5]:
# looking for available date
previous_date = dt.datetime.today() - dt.timedelta(days=1)
prev_day = previous_date.day
dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
dates_objects[prev_day-1].click()

In [6]:
# gettting all available tables
table_data = driver.find_elements(By.XPATH, "//table")

In [7]:
# event
table_data[0].get_attribute('outerHTML')

'<table class="ui-datepicker-calendar"><thead><tr><th scope="col"><span title="Понедельник">Пн</span></th><th scope="col"><span title="Вторник">Вт</span></th><th scope="col"><span title="Среда">Ср</span></th><th scope="col"><span title="Четверг">Чт</span></th><th scope="col"><span title="Пятница">Пт</span></th><th scope="col" class="ui-datepicker-week-end"><span title="Суббота">Сб</span></th><th scope="col" class="ui-datepicker-week-end"><span title="Воскресенье">Вс</span></th></tr></thead><tbody><tr><td class=" " data-handler="selectDay" data-event="click" data-month="4" data-year="2023"><a class="ui-state-default" href="#">1</a></td><td class=" " data-handler="selectDay" data-event="click" data-month="4" data-year="2023"><a class="ui-state-default" href="#">2</a></td><td class=" " data-handler="selectDay" data-event="click" data-month="4" data-year="2023"><a class="ui-state-default" href="#">3</a></td><td class=" " data-handler="selectDay" data-event="click" data-month="4" data-year=

In [8]:
table_data = driver.find_elements(By.XPATH, "//table")

In [9]:
# event data
table_data[1].get_attribute('outerHTML')

'<table class="event"><tbody><tr><td>20.05.2023 12:00   Абедалла Шелбайх - Стефано Травалиа;      (ID=659)</td> <td> Теннис :  ATP. Женева. Квалификация </td> </tr> <tr><td colspan="2">Итог матча:\t    </td></tr> </tbody></table>'

In [10]:
# coefficients
table_data[2].get_attribute('outerHTML')

'<table>   <tbody><tr> <th> Дата-время скана линии </th><th> П1 </th><th> П2 </th><th> 1Х </th><th> 2Х </th><th> Х </th><th> 12 </th><th> ТМ </th><th> ТМ бол </th><th> ТМ мен </th><th> ФОРА 1 </th><th> Коэф Ф1 </th><th> ФОРА 2 </th><th> Коэф Ф2 </th><th> Инд ТОТ 1 </th><th> ИТ1 бол </th><th> ИТ1 мен </th><th> Инд ТОТ 2 </th><th> ИТ2 бол </th><th> ИТ2 мен </th><th> Прим. </th></tr> <tr>  <td>20 май 11:50 </td><td>1,9</td><td>1,83</td><td></td><td></td><td></td><td></td><td>22,5</td><td>1,91</td><td>1,82</td><td>0,5</td><td>1,8</td><td>-0,5</td><td>1,86</td><td>12,5</td><td>1,93</td><td>1,74</td><td>12,5</td><td>1,91</td><td>1,76</td><td></td> </tr>  <tr>  <td>20 май 12:00 </td><td class="c2">1,83</td><td class="c1">1,98</td><td></td><td></td><td></td><td></td><td>22,5</td><td class="c2">1,9</td><td class="c1">1,9</td><td>0,5</td><td>1,8</td><td>-0,5</td><td>1,86</td><td>12,5</td><td class="c1">2</td><td class="c2">1,73</td><td>12,5</td><td class="c1">2</td><td class="c2">1,73</td><td></

In [11]:
# all three tags above are recursively repeated
table_data[3].get_attribute('outerHTML')

'<table class="event"><tbody><tr><td>20.05.2023 12:00   Даниэль Ринкон - Сальваторе Карузо;      (ID=660)</td> <td> Теннис :  ATP. Женева. Квалификация </td> </tr> <tr><td colspan="2">Итог матча:\t    </td></tr> </tbody></table>'

In [12]:
# another table
table_data[-2].get_attribute("outerHTML")

'<table>   <tbody><tr> <th> Дата-время скана линии </th><th> П1 </th><th> П2 </th><th> 1Х </th><th> 2Х </th><th> Х </th><th> 12 </th><th> ТМ </th><th> ТМ бол </th><th> ТМ мен </th><th> ФОРА 1 </th><th> Коэф Ф1 </th><th> ФОРА 2 </th><th> Коэф Ф2 </th><th> Инд ТОТ 1 </th><th> ИТ1 бол </th><th> ИТ1 мен </th><th> Инд ТОТ 2 </th><th> ИТ2 бол </th><th> ИТ2 мен </th><th> Прим. </th></tr> <tr>  <td>20 май 12:00 </td><td>2,57</td><td>1,46</td><td></td><td></td><td></td><td></td><td>20,5</td><td>1,85</td><td>1,85</td><td>3,5</td><td>1,89</td><td>-3,5</td><td>1,81</td><td>11,5</td><td>1,86</td><td>1,8</td><td>12,5</td><td>2,12</td><td>1,61</td><td></td> </tr>  <tr>  <td>20 май 12:30 </td><td class="c1">2,6</td><td class="c2">1,45</td><td></td><td></td><td></td><td></td><td>20,5</td><td>1,85</td><td>1,85</td><td>3,5</td><td class="c1">1,91</td><td>-3,5</td><td class="c2">1,79</td><td>11,5</td><td class="c1">1,88</td><td class="c2">1,78</td><td>12,5</td><td>2,12</td><td>1,61</td><td></td> </tr>  <t

In [13]:
# calendar
table_data[-1].get_attribute("outerHTML")

'<table class="ui-datepicker-calendar"><thead><tr><th scope="col"><span title="Понедельник">Пн</span></th><th scope="col"><span title="Вторник">Вт</span></th><th scope="col"><span title="Среда">Ср</span></th><th scope="col"><span title="Четверг">Чт</span></th><th scope="col"><span title="Пятница">Пт</span></th><th scope="col" class="ui-datepicker-week-end"><span title="Суббота">Сб</span></th><th scope="col" class="ui-datepicker-week-end"><span title="Воскресенье">Вс</span></th></tr></thead><tbody><tr><td class=" " data-handler="selectDay" data-event="click" data-month="4" data-year="2023"><a class="ui-state-default" href="#">1</a></td><td class=" " data-handler="selectDay" data-event="click" data-month="4" data-year="2023"><a class="ui-state-default" href="#">2</a></td><td class=" " data-handler="selectDay" data-event="click" data-month="4" data-year="2023"><a class="ui-state-default" href="#">3</a></td><td class=" " data-handler="selectDay" data-event="click" data-month="4" data-year=

In [14]:
def get_event_data(event_data):
    # Event data
    soup = bs(event_data, 'html.parser')
    tds = soup.find_all('td')
    # date, players
    splitted_str = tds[0].text.split('  ')
    event_date = splitted_str[0].strip()
    event_players = splitted_str[1][:-1].strip()
    # event
    event = tds[1].text.split(' Теннис :  ')[1].strip()
    # result
    try:
        result = tds[2].text.split('\t')[2]
        splitted_result = result.split(', ')
        score = splitted_result[0].strip()
        winner = splitted_result[1][:-1].strip()
    except IndexError:
        score, winner = None, None
    return event_date, event_players, event, score, winner


def get_coefficients_data(table_data):
    # coefficients data 
    soup = bs(table_data, 'html.parser')
    columns = [x.text.strip() for x in soup.find_all('th')]
    table_data = soup.find_all('td')
    rows = []
    for i in range(0, len(table_data), len(columns)):
        row = table_data[i-len(columns):i]
        rows.append(row)
    rows = np.array(rows[1:], dtype=object)
    data_function = np.vectorize(lambda x: x.text.strip())
    rows = data_function(rows)
    df_event = pd.DataFrame(columns=columns, data=rows)
    nan_value = float("NaN")
    df_event.replace("", nan_value, inplace=True)
    df_event.dropna(how='all', axis=1, inplace=True)
    return df_event

In [15]:
# one page parsing
df_events = pd.DataFrame()
dict_odds = {}
events = []
for table in table_data:
    id_ = str(uuid.uuid4())[:8]
    table_html = table.get_attribute('outerHTML')
    if 'class="event"' in table_html:
        events.append(tuple([id_])+get_event_data(table_html))
    if 'Дата-время скана линии' in table_html :
        dict_odds[id_] = get_coefficients_data(table_html)
df_events = pd.DataFrame(
    columns=['id', 'date', 'players', 'event', 'score', 'winner'],
    data=events
)

In [24]:
bc_btn[0].click()

In [27]:
df_events.head()

,id,date,players,event,score,winner
0,e133d899,20.05.2023 12:00,Абедалла Шелбайх - Стефано Травалиа,ATP. Женева. Квалификация,None,None
1,5da7fd41,20.05.2023 12:00,Даниэль Ринкон - Сальваторе Карузо,ATP. Женева. Квалификация,None,None
2,304fa5be,20.05.2023 12:00,Тьяго Моура Монтейро - Фернандо Вердаско,ATP. Женева. Квалификация,"Счет: 2:0 (6:2,6:2)",Победа: Тьяго Моура Монтейро
3,297950e8,20.05.2023 12:00,Тьяго Моура Монтейро - Фернандо Вердаско,ATP. Женева. Квалификация,None,None
4,c144282a,20.05.2023 13:30,Артур Казо - Джером Кьюм,ATP. Женева. Квалификация,"Счет: 2:1 (7:6,6:7,6:4)",Победа: Артур Казо


dict_allodds: bc -> odds<br>
dict_allevents: bc -> events

In [89]:
# one page parser, all bcs
retry_number = 5
bcs = ["tab_1x", "tab_bc", "tab_fb", "tab_mf"]
dict_allevents = {bc: pd.DataFrame() for bc in bcs}
dict_allodds = {bc: dict() for bc in bcs}
for bc in tqdm(bcs, 'bc', position=0):
    bc_btn = driver.find_elements(By.ID, bc)
    assert len(bc_btn) > 0, f'{bc} is not available'
    bc_btn[0].click()
    # one page parsing
    dict_odds = {}
    events = []
    table_data = driver.find_elements(By.XPATH, "//table")
    for table in tqdm(table_data, 'table', position=1, leave=False):
        id_ = str(uuid.uuid4())[:8]
        for i in range(retry_number):
            try:
                table_html = table.get_attribute('outerHTML')
            except StaleElementReferenceException:
                pass
            else:
                break 
        if 'class="event"' in table_html:
            events.append(tuple([id_])+get_event_data(table_html))
        if 'Дата-время скана линии' in table_html :
            dict_odds[id_] = get_coefficients_data(table_html)
            dict_allodds[bc][id_] = dict_odds[id_]
    df_events = pd.DataFrame(
        columns=['id', 'date', 'players', 'event', 'score', 'winner'],
        data=events
    )
    dict_allevents[bc] = pd.concat((dict_allevents[bc], df_events))

bc: 100%|██████████| 4/4 [00:54<00:00, 13.54s/it]


In [16]:
# # Event data
# event_data = table_data[1].get_attribute('outerHTML')
# soup = bs(event_data, 'html.parser')
# tds = soup.find_all('td')

# # date, players
# splitted_str = tds[0].text.split('  ')
# event_date = splitted_str[0].strip()
# event_players = splitted_str[1][:-1].strip()

# # event
# event = tds[1].text.split(' Теннис :  ')[1].strip()

# # result
# result = tds[2].text.split('\t')[2]
# splitted_result = result.split(', ')
# score = splitted_result[0].strip()
# winner = splitted_result[1][:-1].strip()

In [17]:
# # coefficients data 
# table = table_data[2].get_attribute('outerHTML')
# soup = bs(table, 'html.parser')
# columns = [x.text.strip() for x in soup.find_all('th')]
# table_data = soup.find_all('td')
# rows = []
# for i in range(0, len(table_data), len(columns)):
#     row = table_data[i-len(columns):i]
#     rows.append(row)
# rows = np.array(rows[1:], dtype=object)
# data_function = np.vectorize(lambda x: x.text.strip())
# rows = data_function(rows)
# df_event = pd.DataFrame(columns=columns, data=rows)
# nan_value = float("NaN")
# df_event.replace("", nan_value, inplace=True)
# df_event.dropna(how='all', axis=1, inplace=True)